# 🧠 MODÈLE PRÉDICTIF - SILENT SURGE
# **Objectif** : Prédire les best-sellers avec interprétabilité

In [11]:
try:
    # Core
    import pandas as pd
    import numpy as np
    from pathlib import Path
    
    # ML
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import make_pipeline
    
    # Visualisation (version sécurisée)
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.style.use('ggplot')  # Style alternatif plus stable
    
    # Utilitaires
    import joblib
    
    print("✅ Tous les imports réussis")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print("Solution : lancez dans le terminal :")
    print("pip install pandas scikit-learn matplotlib seaborn joblib")

✅ Tous les imports réussis


## 3. PRÉPARATION DES DONNÉES

In [13]:
# Features et Target
features = ["saison", "zone", "type_commerce"]
target = "produit"

# Encodage intelligent (évite le LabelEncoder problématique)
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)],
    remainder='passthrough'
)

X = df[features]
y = df[target]

# Split stratifié
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [14]:

# 1. IMPORTS OBLIGATOIRES
import pandas as pd
from pathlib import Path
import sys

# Vérification des imports critiques
try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import make_pipeline
    import joblib
    print("✅ Import ML réussis")
except ImportError:
    !{sys.executable} -m pip install scikit-learn joblib
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import make_pipeline
    import joblib


# 2. CHARGEMENT DES DONNÉES

print("Étape 1: Chargement des données...")

# Chemin absolu vérifié
DATA_PATH = Path(r"C:\Users\Sajed\Desktop\SilentSurgeFinal\data")

# Liste des fichiers requis
FICHIERS = {
    'pizzeria': DATA_PATH / "ventes_pizzeria.csv",
    'boucherie': DATA_PATH / "ventes_boucherie.csv",
    'epicerie': DATA_PATH / "ventes_epicerie.csv"
}

# Vérification visuelle
print("\nContenu du dossier data/:")
for f in DATA_PATH.glob("*.csv"):
    print(f"- {f.name}")

# Chargement avec vérification
dfs = []
for nom, chemin in FICHIERS.items():
    try:
        df_temp = pd.read_csv(chemin).assign(type_commerce=nom.upper())
        dfs.append(df_temp)
        print(f"\n✅ {nom} chargé : {len(df_temp)} lignes")
    except Exception as e:
        print(f"\n❌ Erreur sur {nom} : {str(e)}")
        continue

if not dfs:
    raise ValueError("Aucun fichier chargé - vérifiez les données")

df = pd.concat(dfs, ignore_index=True)
print(f"\nDonnées finales : {len(df)} lignes")


# 3. PRÉPARATION DES DONNÉES
print("\nÉtape 2: Préparation des données...")

features = ["saison", "zone", "type_commerce"]
target = "produit"

# Vérification des colonnes
colonnes_manquantes = [col for col in features + [target] if col not in df.columns]
if colonnes_manquantes:
    raise ValueError(f"Colonnes manquantes : {colonnes_manquantes}")

# Préprocessing
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)],
    remainder='passthrough'
)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    test_size=0.2,
    stratify=df[target],
    random_state=42
)


# 4. MODÉLISATION
print("\nÉtape 3: Entraînement du modèle...")

model = make_pipeline(
    preprocessor,
    RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        class_weight='balanced'
    )
)

model.fit(X_train, y_train)
print("✅ Modèle entraîné")

# Sauvegarde
Path("models").mkdir(exist_ok=True)
joblib.dump(model, "models/modele_prod.joblib")
print("💾 Modèle sauvegardé")


# 5. ÉVALUATION
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

print("\n📊 Performance du modèle:")
print(classification_report(y_test, y_pred))
print(f"Précision globale: {accuracy_score(y_test, y_pred):.2%}")

✅ Import ML réussis
Étape 1: Chargement des données...

Contenu du dossier data/:
- ventes_boucherie.csv
- ventes_epicerie.csv
- ventes_pizzeria.csv

✅ pizzeria chargé : 1646 lignes

✅ boucherie chargé : 1695 lignes

✅ epicerie chargé : 1668 lignes

Données finales : 5009 lignes

Étape 2: Préparation des données...

Étape 3: Entraînement du modèle...
✅ Modèle entraîné
💾 Modèle sauvegardé

📊 Performance du modèle:
                   precision    recall  f1-score   support

       4 fromages       0.00      0.00      0.00        10
  Blanc de poulet       0.13      0.11      0.12        35
       Brochettes       0.08      0.03      0.05        59
          Calzone       0.10      0.18      0.13        11
            Chips       0.15      0.18      0.16        33
             Coca       0.00      0.00      0.00        37
Cuisses de canard       0.00      0.00      0.00        37
            Curly       0.00      0.00      0.00        35
     Côte de bœuf       0.03      0.14      0.05   

c:\Users\Sajed\Desktop\SilentSurgeFinal\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Sajed\Desktop\SilentSurgeFinal\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Sajed\Desktop\SilentSurgeFinal\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi